# Introduction to Ray Core (Advancement): Object store, Tasks, Actors

© 2025, Anyscale. All Rights Reserved

💻 **Launch Locally**: You can run this notebook locally.

🚀 **Launch on Cloud**: Think about running this notebook on a Ray Cluster (Click [here](http://console.anyscale.com/register) to easily start a Ray cluster on Anyscale)

This notebook provides a step-by-step introduction to Object store, Tasks, and Actors, which are all the fundamental building blocks of Ray that enables distributed computing.

<div class="alert alert-block alert-info">

<b> Here is the roadmap for this notebook </b>

<ol>
  <li>Object store
    <ul>
      <li>Pattern: pass an object as a top-level argument</li>
    </ul>
  </li>
  <li>Chaining Tasks and Passing Data</li>
  <li>Task retries</li>
  <li>Task Runtime Environments
    <ul>
      <li>Note about pip dependencies</li>
    </ul>
  </li>
  <li>Resource allocation and management
    <ul>
      <li>Note on resources requests, available resources, configuring large clusters</li>
      <li>Fractional resources</li>
      <li>IO bound tasks and fractional resources</li>
    </ul>
  </li>
  <li>Nested Tasks</li>
  <li>Pattern: Pipeline data processing and waiting for results
    <ul>
      <li>Batch Processing Pattern</li>
      <li>Note on fetching too many objects at once with ray.get causes failure</li>
    </ul>
  </li>
  <li>Ray Actors</li>
</ol>
</div>

**Imports**

In [ ]:
import os
import random
import sys
import time

import numpy as np
import ray

## 1. Object store

Each worker node has its own object store, and collectively, these form a shared object store across the cluster.

Remote objects are immutable. That is, their values cannot be changed after creation. This allows remote objects to be replicated in multiple object stores without needing to synchronize the copies.

|<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/ray-core/ray-cluster.png" width="70%" loading="lazy">|
|:--|
|A Ray cluster with a head node and two worker nodes. Highlighted in orange is distributed object store.|

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-core/key-concepts.html#objects" target="_blank">Object</a></strong> - tasks and actors create and work with remote objects, which can be stored anywhere in a cluster. These objects are accessed using <strong>ObjectRef</strong> and are cached in a distributed shared-memory <strong>object store</strong>.
</div>

Let's consider following example:

In [ ]:
large_matrix = np.random.rand(1024, 1024, 1024//8) # approx. 1 GB
size_in_bytes = sys.getsizeof(large_matrix)

print(f"large_matrix has: {size_in_bytes/1024/1024/1024:.2f} GB")

Add an object to the object store using `ray.put()`

In [ ]:
obj_ref = ray.put(large_matrix)
obj_ref

Use the `ray.get()` method to fetch the result of a remote object from an object ref

In [ ]:
large_mat_from_object_store = ray.get(obj_ref)

In [ ]:
np.array_equal(large_mat_from_object_store, large_matrix)

In [ ]:
large_mat_from_object_store is large_matrix

### 1.1. Pattern: pass an object as a top-level argument

When an object is passed directly as a top-level argument to a task, Ray will de-reference the object. This means that Ray will fetch the underlying data for all top-level object reference arguments, not executing the task until the object data becomes fully available.

<div class="alert alert-info">
This pattern assumes that two conditions are satisfied:
<ol>
<li> the object is large</li>
<li> user wants to reuse the object multiple times</li>
</ol>
</div>

In [ ]:
@ray.remote
def compute(x, y):
    return int(np.matmul(x, y).sum())

In [ ]:
mat1_ref = ray.put(np.random.rand(32, 32))
mat2_ref = ray.put(np.random.rand(32, 32))

In [ ]:
collection = []
for i in range(10):
    collection.append(compute.remote(mat1_ref, mat2_ref))

results = ray.get(collection)
results

## 2. Chaining Tasks and Passing Data

Let's say we now want to execute a graph of two tasks:
1. Square a value using `expensive_square`
2. Add 1 to the `expensive_square` result, by using `remote_add`

This can be achieved without fetching an intermediate result.

Anti-pattern:

In [ ]:
@ray.remote
def remote_add(a, b):
    return a + b

@ray.remote
def expensive_square(x):
    time.sleep(5)
    return x**2

# 1st task
square_ref = expensive_square.remote(2)
square_value = ray.get(square_ref)

# 2nd task
sum_ref = remote_add.remote(1, square_value)
sum_value = ray.get(sum_ref)

Chain the tasks by passing the `ObjectRef` directly to the second task:

In [ ]:
square_ref = expensive_square.remote(2)
sum_ref = remote_add.remote(1, square_ref)
sum_value = ray.get(sum_ref)

In this way Ray doesn't fetch data to the "driver" process, *especially* if the returned object is large.

The term "driver" refers to the process that initiated the connection to the cluster which in this case is the Python process running this notebook.

Under the hood, Ray will still call `ray.get` on the first task

i.e. effectively, Ray will do something like this to make the argument available to the second task:

```python
def expensive_square(x):
    if isinstance(x, ObjectRef):
        x = ray.get(x)
    time.sleep(5)
    return x**2
```

The benefit of this approach is that data at most needs to be transferred once between the first and second task. Instead of going through the driver process. To read more about this, see [Passing object arguments](https://docs.ray.io/en/latest/ray-core/objects.html#passing-object-arguments).

Also note, you can bypass this behavior by wrapping/nesting the object ref in a container object (e.g., a tuple, list, or dict):

In [ ]:
ref = expensive_square.remote(1)
out_ref = remote_add.remote([ref], [ref])
ray.get(out_ref) 

## 3. Task retries

Let's consider two types of exceptions:
1. **system errors** (e.g., Python-level exceptions)
2. **application-level errors** (e.g., a machine fails)

Ray will automatically **retry a task up to 3 times**, if it fails due to a system error (e.g., a worker node dies).

Below task won't be retried by default because it's an application failure

In [ ]:
@ray.remote
def incorrect_square(x: int, prob: float) -> int:
    # Simulate potential failures
    if random.random() < prob:  # % chance of failure
        raise ValueError("Random failure")
    return x**2

In [ ]:
try:
    ray.get([incorrect_square.remote(x=4, prob=0.5) for _ in range(10)])
except ray.exceptions.RayTaskError:
    print("At least one of the tasks failed", flush=True)

Ray let's you specify how to handle retries when an exception is encountered.

Let's retry on `ValueError`, like below:

In [ ]:
@ray.remote(retry_exceptions=[ValueError])
def correct_square(x: int, prob: float) -> int:
    # Simulate potential failures
    if random.random() < prob:  # % chance of failure
        raise ValueError("Random failure")

    return x**2

Note we did not have to re-define the remote function, instead we could have used `.options`

In [ ]:
correct_square_mod = correct_square.options(
    retry_exceptions=[ValueError], max_retries=10,
)

Let's try it out:

In [ ]:
try:
    outputs = ray.get([correct_square_mod.remote(x=4, prob=0.5) for _ in range(10)])
except ray.exceptions.RayTaskError:
    print("At least one of the tasks failed", flush=True)

outputs

<div class="alert alert-info">
Refer to the <strong><a href="https://docs.ray.io/en/latest/ray-core/tasks/retries.html" target="_blank">retries</a></strong> to learn more.
</div>

## 4. Task Runtime Environments

Runtime environments can be used on top of the prepared environment from the Ray Cluster to customize the execution of tasks.

When setting up a worker process to run a task, Ray will first prepare the environment for the task.

This includes things like:
* installing dependencies
* setting environment variables

For example, we can set an environment variable:

In [ ]:
@ray.remote(runtime_env={"env_vars": {"MY_CUSTOM_ENV": "prod"}})
def f():
    env = os.environ["MY_CUSTOM_ENV"]
    return f"My custom env is {env}"

In [ ]:
ray.get(f.remote())

### 4.1. Note about pip dependencies

pip dependencies in task runtime environments don't come for free. They add to the startup time of the worker process.

If you find yourself needing to install the same dependencies across many tasks, consider baking them into the image you use to start your Ray cluster.

## 5. Resource allocation and management

By default, Ray will schedule a task as long as there is at least one CPU available.

In code this can be specified in the `ray.remote`, like this:

In [ ]:
@ray.remote(num_cpus=1)
def remote_add(a, b):
    return a + b

However, these resource specifications are not enforced - i.e. they are entirely [logical and not physical](https://docs.ray.io/en/latest/ray-core/scheduling/resources.html#physical-resources-vs-logical-resources).

This means that you can for instance perform multiprocessing or multithreading within a task and oversubscribe to resources.

In [ ]:
@ray.remote(num_cpus=1)
def mm(n: int = 4000):
    A = np.random.rand(n, n)
    B = np.random.rand(n, n)

    # Time the dot product
    start = time.time()
    C = np.dot(A, B)
    end = time.time()
    print(f"Took {end - start}s")
    
ray.get(mm.options(runtime_env={"env_vars": {"OMP_NUM_THREADS": "1"}}).remote())
ray.get(mm.options(runtime_env={"env_vars": {"OMP_NUM_THREADS": "8"}}).remote())

<div class="alert alert-info">

Note by default, Ray will set the `OMP_NUM_THREADS` environment variable to the number of CPUs in the cluster.

Learn more about <strong><a href="https://docs.ray.io/en/latest/ray-core/scheduling/resources.html#physical-resources-and-logical-resources" target="_blank">physical resources and logical resources</a></strong>.
</div>

### 5.1. Note on resources requests, available resources, configuring large clusters

<p>During the <em>scheduling stage</em>, Ray evaluates the <strong>resource requirements</strong> specified via the <code>@ray.remote</code> decorator or within the <code>resources={...}</code> argument. These requirements may include:</p>

<ul>
    <li><strong>CPU</strong> e.g., <code>@ray.remote(num_cpus=2)</code>)</li>
    <li><strong>GPU</strong> e.g., <code>@ray.remote(num_gpus=1)</code>)</li>
    <li><strong>Custom resources</strong>: User-defined custom resources like <code>"TPU"</code></li>
    <li><strong>Memory</strong></li>
</ul>

<p>Ray's scheduler checks the <strong>resource specification</strong> (sometimes referred to as <strong>resource shape</strong>) to match tasks and actors with available resources in the cluster. If the exact resource combination is unavailable, Ray may autoscale the cluster.</p>

<p>You can inspect the current resource availability using:</p>
<pre><code>
ray.available_resources()
</code></pre>

<p>This returns a dictionary showing the currently available CPUs, GPUs, memory, and any custom resources, for example:</p>

<pre><code>{'CPU': 24.0, 'GPU': 1.0, 'memory': 2147483648.0}</code></pre>

In [ ]:
ray.available_resources()

<div class="alert alert-info">

<strong>Pattern:</strong> configure the head node to be unavailable for compute tasks.

When scaling to large clusters, it's important to ensure that the <strong>head node</strong> does not handle any compute tasks. Users can indicate that the head node is unavailable for compute by setting its resources:

```resources: {"CPU": 0}```

Learn more about <strong><a href="https://docs.ray.io/en/latest/cluster/vms/user-guides/large-cluster-best-practices.html#configuring-the-head-node" target="_blank">configuring the head node</a></strong>.
</div>

### 5.2. Fractional resources

Fractional resources allow Ray Tasks to request a fraction of a CPU or GPU (e.g., 0.5), enabling finer-grained resource allocation.

Let's consider the above example again:

In [ ]:
@ray.remote(num_cpus=0.5)
def remote_add(a, b):
    return a + b

In [ ]:
ref = remote_add.remote(2, 3)
ref

In [ ]:
ray.get(ref)

<div class="alert alert-info">
    Fractional resources include support for <strong><a href="https://docs.ray.io/en/latest/ray-core/scheduling/accelerators.html#fractional-accelerators" target="_blank">multiple accelerators</a></strong>, allowing users to load multiple smaller models onto a single GPU. This is especially useful for scenarios like batch inference. Learn more about <strong><a href="https://docs.ray.io/en/latest/ray-core/scheduling/resources.html#fractional-resource-requirements" target="_blank">fractional resource requirements</a></strong>.
</div>

### 5.3. IO bound tasks and fractional resources

Setting fractional cpus or even `num_cpus=0` is a pattern for <strong>I/O-bound tasks</strong> that do not require CPU-intensive computation.

<p>This allows Ray to oversubscribe CPUs, scheduling many such tasks concurrently without reserving CPU cores. Since <code>num_cpus=0</code> always passes the scheduler’s resource check, these tasks get scheduled immediately.</p>

<p>This can lead to <strong>resource savings</strong> and better utilization in workloads with high I/O.</p>

<p>More details about <a href="https://docs.ray.io/en/latest/ray-core/scheduling/resources.html#fractional-resource-requirements" target="_blank">
fractional resource requirements</a>.</p>

## 6. Nested Tasks

Ray __does not__ require that all of your tasks and their dependencies be arranged from one "driver" process.

For example, you can have a main task that schedules other tasks and then waits for them to complete.

In [ ]:
@ray.remote
def main():
    square_ref_1 = expensive_square.remote(1)
    square_ref_2 = expensive_square.remote(2)
    add_ref = remote_add.remote(square_ref_1, square_ref_2)
    return ray.get(add_ref)

In [ ]:
ray.get(main.remote())

In this example:
1. Our local process requests Ray to schedule a `main` task in the cluster
2. Ray executes the `main` task in a separate worker process
3. Inside `main`, we invoke multiple `expensive_square` tasks, which Ray distributes across available worker processes
4. Once all "sub tasks" complete, `main` returns the final value

This ability for tasks to schedule other tasks using uniform semantics makes Ray particularly powerful and flexible.

<div class="alert alert-info">
To avoid deadlocks, Ray yields CPU resources while blocked waiting for a task to complete. Read more about <strong><a href="https://docs.ray.io/en/latest/ray-core/tasks/nested-tasks.html#yielding-resources-while-blocked" target="_blank">yielding resources while blocked</a></strong>.
</div>

## 7. Pattern: Pipeline data processing and waiting for results

After launching a number of tasks, you may want to know which ones have finished executing without blocking on all of them. This could be achieved by `ray.wait()`

|<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/ray-core/pipeline-data-processing.png" width="50%" loading="lazy">|
|:--|
|(top panel) Execution timeline when using ray.get() to wait for all results before calling process results. (bottom panel) Execution timeline when using ray.wait() to process results as soon as they become available.|

Let's modify `expensive_square` a bit:

In [ ]:
@ray.remote
def expensive_square(x):
    time.sleep(np.random.randint(1, 10))
    return x**2

In [ ]:
expensive_compute = []

for i in range(15):
    expensive_compute.append(expensive_square.remote(i))

expensive_compute

Process items as soon as they become available

In [ ]:
ready_refs, not_ready_refs = ray.wait(expensive_compute) # wait for next object ref that is ready

# process new item as soon as it becomes available
while not_ready_refs:
    print(f"{ready_refs[0]} is ready; result: {ray.get(ready_refs[0])}")
    print(f"{len(not_ready_refs)} items not ready... \n")

    ready_refs, not_ready_refs = ray.wait(not_ready_refs) # wait for next item

    assert len(ready_refs) == 1, f"len(ready_refs) should be 1, got {len(ready_refs)} instead"

print(f"I'm the last item: {ready_refs[0]}; result: {ray.get(ready_refs[0])}")

<div class="alert alert-info">
Read more about the <strong><a href="https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html#tip-4-pipeline-data-processing" target="_blank">pipeline data processing</a></strong>
</div>

### 7.1 Batch Processing Pattern

Program can wait for a batch of `ObjectRef`, before returning. Let's consider this scenario:

In [ ]:
expensive_compute = []

for i in range(15):
    expensive_compute.append(expensive_square.remote(i))

expensive_compute

In [ ]:
BATCH_SIZE = 3

ready_refs, not_ready_refs = ray.wait(expensive_compute, num_returns=BATCH_SIZE)  # wait for BATCH_SIZE object refs

# process new item as soon as it becomes available
while not_ready_refs:
    print(f"{ready_refs} are ready; results: {ray.get(ready_refs)}")
    print(f"{len(not_ready_refs)} items not ready... \n")
    ready_refs, not_ready_refs = ray.wait(not_ready_refs, num_returns=BATCH_SIZE)  # wait for BATCH_SIZE object refs

print(f"Last batch {ready_refs}; result: {ray.get(ready_refs)}")

### 7.2 Note on fetching too many objects at once with ray.get causes failure

Calling `ray.get()` on too many objects will lead to **heap out-of-memory** or **object store out-of-space**.

```python
object_refs = [expensive_square.remote(i) for i in range(1_000_000)]

all_results_at_once = ray.get(object_refs)
all_results_at_once
```

Instead fetch and process one batch at a time.

<div class="alert alert-info">
Read more about this <strong><a href="https://docs.ray.io/en/latest/ray-core/patterns/ray-get-too-many-objects.html#anti-pattern-fetching-too-many-objects-at-once-with-ray-get-causes-failure" target="_blank">anti-pattern</a></strong>.
</div>

## 8. Ray Actors

Actors extend the Ray API from functions (tasks) to classes.

An actor is a stateful worker. When a new actor is instantiated, a new worker is created, and methods of the actor are scheduled on that specific worker and can access and mutate the state of that worker. Similarly to Ray Tasks, actors support CPU and GPU compute as well as fractional resources.

Let's look at an example of an actor which maintains a running balance.

In [ ]:
@ray.remote
class Accounting:
    def __init__(self):
        self.total = 0
    
    def add(self, amount):
        self.total += amount
        
    def remove(self, amount):
        self.total -= amount
        
    def total(self):
        return self.total

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-core/key-concepts.html#actors" target="_blank">Actor</a></strong> is a remote, stateful Python class.
</div>

<div class="alert alert-info">

The most common use case for actors is with state that is not mutated but is large enough that we may want to load it only once and ensure we can route calls to it over time, such as a large AI model.

</div>

Define an actor with the `@ray.remote` decorator and then use `<class_name>.remote()` to ask Ray to construct an instance of this actor somewhere in the cluster.

We get an actor handle which we can use to communicate with that actor, pass to other code, tasks, or actors, etc.

In [ ]:
acc = Accounting.remote()

We can send a message to an actor -- with RPC semantics -- by using `<handle>.<method_name>.remote()`

In [ ]:
acc.total.remote()

Not surprisingly, we get an object ref back

In [ ]:
ray.get(acc.total.remote())

We can mutate the state inside this actor instance

In [ ]:
acc.add.remote(100)

In [ ]:
acc.remove.remote(10)

In [ ]:
ray.get(acc.total.remote())

<div class="alert alert-block alert-info">

__Activity: linear model inference__

* Create an actor which applies a model to convert Celsius temperatures to Fahrenheit
* The constructor should take model weights (w1 and w0) and store them as instance state
* A convert method should take a scalar, multiply it by w1 then add w0 (weights retrieved from instance state) and then return the result

```python

# Hint: define the below as a remote actor
class LinearModel:
    def __init__(self, w0, w1):
        # Hint: store the weights

    def convert(self, celsius):
        # Hint: convert the celsius temperature to Fahrenheit

# Hint: create an instance of the LinearModel actor

# Hint: convert 100 Celsius to Fahrenheit
```

</div>

In [ ]:
# Write your solution here

<div class="alert alert-block alert-info">

<details>

<summary> Click to see solution </summary>

```python
@ray.remote
class LinearModel:
    def __init__(self, w0, w1):
        self.w0 = w0
        self.w1 = w1

    def convert(self, celsius):
        return self.w1 * celsius + self.w0

model = LinearModel.remote(w1=9/5, w0=32)
ray.get(model.convert.remote(100))
```

</details>
</div>


<!-- TODO: add Patterns/antipatterns based on above learnings-->
